In [120]:
#rimas - 439808955

In [121]:
!pip install pyspark


In [122]:
#import :

# Import necessary libraries

import pyspark

In [123]:
from pyspark.sql import SparkSession
## Create a SparkSession


spark = SparkSession.builder.appName("py spark").config("spark.some.config.option", "some-value").getOrCreate()

In [124]:
#load dataset:"# Show the schema and content of the DataFrame"
csv_file="/content/Advertising.csv"

df=spark.read.option("header","true").option("inferSchema","true").csv(csv_file)

In [125]:
#show schema + content :
df.printSchema()

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



In [126]:
df.show() #content 

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|  8.7| 48.9|     75.0|  7.2|
| 57.5| 32.8|     23.5| 11.8|
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|      1.0|  4.8|
|199.8|  2.6|     21.2| 10.6|
| 66.1|  5.8|     24.2|  8.6|
|214.7| 24.0|      4.0| 17.4|
| 23.8| 35.1|     65.9|  9.2|
| 97.5|  7.6|      7.2|  9.7|
|204.1| 32.9|     46.0| 19.0|
|195.4| 47.7|     52.9| 22.4|
| 67.8| 36.6|    114.0| 12.5|
|281.4| 39.6|     55.8| 24.4|
| 69.2| 20.5|     18.3| 11.3|
|147.3| 23.9|     19.1| 14.6|
+-----+-----+---------+-----+
only showing top 20 rows



In [127]:
df.describe().show() #describe

+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|               0.0|               0.3|               1.6|
|    max|            296.4|              49.6|             114.0|              27.0|
+-------+-----------------+------------------+------------------+------------------+



In [128]:
from pyspark.sql import Row
# from pyspark.ml.linalg import Vector





In [129]:
from pyspark.ml.linalg import Vectors
# Define the transData function
3# Define a function to transform the data

def transData(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]), r[-1]]).toDF(["features", "label"]) #good with big data 

In [130]:
## Import necessary libraries for machine learning pipeline

from pyspark.ml.linalg import Vectors

In [131]:

# Transform the data
transform = transData(df)
transform.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[230.1,37.8,69.2]| 22.1|
| [44.5,39.3,45.1]| 10.4|
| [17.2,45.9,69.3]|  9.3|
|[151.5,41.3,58.5]| 18.5|
|[180.8,10.8,58.4]| 12.9|
+-----------------+-----+
only showing top 5 rows



In [132]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

transform = transData(df)

from pyspark.ml.feature import VectorIndexer

from pyspark.ml.feature import VectorIndexer

# Create a VectorIndexer to encode the features

# Create the VectorIndexer
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(transform) #convort categre to -> numrcal


data=featureIndexer.transform(transform)

# featureIndexer=VectorIndexer(inputCol="features",outputCol="indexedFeatures",maxCategories=4).fit(transform)

In [133]:
data.show(5) # nothing change in num becose in all numrcal

+-----------------+-----+-----------------+
|         features|label|  indexedFeatures|
+-----------------+-----+-----------------+
|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----------------+-----+-----------------+
only showing top 5 rows



In [134]:
#•	Use Pysaprck and MLlib packages to train
# and evaluate a regression model on the advertising dataset (3.25 marks).

# Split the transformed data into training and testing sets


(trainingData, testData)=transform.randomSplit([0.6,0.4])






In [135]:
trainingData.show(5)

+----------------+-----+
|        features|label|
+----------------+-----+
|  [0.7,39.6,8.7]|  1.6|
| [7.8,38.9,50.6]|  6.6|
|  [8.4,27.2,2.1]|  5.7|
|[11.7,36.9,45.2]|  7.3|
| [13.1,0.4,25.6]|  5.3|
+----------------+-----+
only showing top 5 rows



In [136]:
testData.show(5)

+---------------+-----+
|       features|label|
+---------------+-----+
| [4.1,11.6,5.7]|  3.2|
| [5.4,29.9,9.4]|  5.3|
|[7.3,28.1,41.4]|  5.5|
|  [8.6,2.1,1.0]|  4.8|
|[8.7,48.9,75.0]|  7.2|
+---------------+-----+
only showing top 5 rows



In [137]:
#fit : model

from pyspark.ml.regression import LinearRegression

# Create the LinearRegression model
lr=LinearRegression()

In [138]:
#pipeline :  stages
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Create the Pipeline
# Create a Pipeline with VectorIndexer and LinearRegression

pipeline = Pipeline(stages=[featureIndexer, lr])

# Fit the pipeline to the training data
model = pipeline.fit(trainingData)




In [139]:
# def modelsummary(model):
#     import numpy as np
#     print("Note: the last rows are the information for Intercept")
#     print("##", "Estimate | Std. Error | t Values | P-value")

#     coef = np.append(list(model.coefficients), model.intercept)
#     Summary = model.stages[-1].summary

#     for i in range(len(Summary.pValues)):
#         print("##", '---')
#         print("##", '{:10.6f}'.format(coef[i]), '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
#               '{:8.3f}'.format(Summary.tValues[i]), '{:10.6f}'.format(Summary.pValues[i]))

#     print("##", '---')
#     print("##", "Mean squared error: %.6f" % Summary.meanSquaredError, ", RMSE: %.6f" % Summary.rootMeanSquaredError)
#     print("##", "Multiple R-squared: %f" % Summary.r2, ", Total iterations: %i" % Summary.totalIterations)

In [140]:
# # Define a function to print model summary

# def modelSummary(model):
#     import numpy as np

#     print("note: last rows are the info for the intercept")
#     print("#########################################")
#     print("## , estimate, std, value, p-value")

#     coef = np.append(list(model.coefficients), model.intercept)
#     Summary = model.summary

#     for i in range(len(Summary.pValues)):
#         print("## ", '{:10.6f}'.format(coef[i]), \
#               '{:10.6f}'.format(Summary.coefficientStandardErrors[i]), \
#               '{:8.3f}'.format(Summary.tValues[i]), \
#               '{:10.6f}'.format(Summary.pValues[i]))

In [141]:
# linearRegressionModel = model.stages[-1]
# # Print the model summary


# print("Mean Squared Error (MSE):", linearRegressionModel.Summary.meanSquaredError)
# print("Root Mean Squared Error (RMSE):", linearRegressionModel.Summary.rootMeanSquaredError)
# print("R-squared:", linearRegressionModel.Summary.r2)
# print("Total Iterations:", linearRegressionModel.Summary.totalIterations)

In [142]:
def modelsummary(model): #evluation :
    import numpy as np
    print("Note: the last rows are the information for Intercept")
    print("##", "Estimate | Std. Error | t Values | P-value")

    coef = np.append(list(model.coefficients), model.intercept)
    Summary = model.summary

    for i in range(len(Summary.pValues)):
        print("##", '---')
        print("##", '{:10.6f}'.format(coef[i]), '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
              '{:8.3f}'.format(Summary.tValues[i]), '{:10.6f}'.format(Summary.pValues[i]))

    print("##", '---')
    print("##", "Mean squared error: %.6f" % Summary.meanSquaredError, ", RMSE: %.6f" % Summary.rootMeanSquaredError)
    print("##", "Multiple R-squared: %f" % Summary.r2, ", Total iterations: %i" % Summary.totalIterations)

In [143]:
modelsummary(model.stages[-1]) # Print the model summary


Note: the last rows are the information for Intercept
## Estimate | Std. Error | t Values | P-value
## ---
##   0.045207   0.002027   22.299   0.000000
## ---
##   0.192532   0.011811   16.301   0.000000
## ---
##   0.000014   0.007941    0.002   0.998632
## ---
##   2.931912   0.459943    6.375   0.000000
## ---
## Mean squared error: 2.923414 , RMSE: 1.709799
## Multiple R-squared: 0.888405 , Total iterations: 0


In [144]:
#prediction :

# Make predictions on the test data

predictions = model.transform(testData)

In [145]:
predictions.select("features", "label", "prediction").show(5)

+---------------+-----+------------------+
|       features|label|        prediction|
+---------------+-----+------------------+
| [4.1,11.6,5.7]|  3.2| 5.350703718047535|
| [5.4,29.9,9.4]|  5.3| 8.932850545788323|
|[7.3,28.1,41.4]|  5.5| 8.672623160163077|
|  [8.6,2.1,1.0]|  4.8| 3.725019728974671|
|[8.7,48.9,75.0]|  7.2|12.741027659716588|
+---------------+-----+------------------+
only showing top 5 rows



In [146]:
#evluation :
# Evaluate the model using RMSE

from pyspark.ml.evaluation import RegressionEvaluator

evaluator= RegressionEvaluator(labelCol="label",predictionCol="prediction",metricName="rmse")


rmse=evaluator.evaluate(predictions)

print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 1.6192434923667036


In [147]:
# Calculate R-squared score

y_t=predictions.select("label").toPandas()
y_p=predictions.select("prediction").toPandas()


import sklearn.metrics
r2_score=sklearn.metrics.r2_score(y_t,y_p)

print("r2-score" , r2_score)

r2-score 0.9071181316359046


In [148]:
spark.stop()# Stop the SparkSession


In [1]:
#•	Comment your codes and show clearly the steps of your solution (0.75 mark).   

print("done")


done


In [151]:
#•	Explain to which extent your model is good and explainable (1 marks).

print("""
The goal of the code is to build a machine learning model to predict sales and thus to an advertising data set.


Mean squared error (MSE): 2.3048322297869657
Root Mean Squared Error (RMSE): 1.5181673918863379
R-squared (R2-score): 0.9169424999582725
Total iterations: 0

These results indicate that the model has a good ability to explain the variance in sales data,
explaining about 91.7% of the variance in the data (R-squared).

There is a noticeable difference between the MSE value and the RMSE value,
which indicates that the sales forecast provided by the model is inaccurate.

---------------------------------------------------------------more detel-----------------------------------


1. The MSE value is 2.3048322297869657, indicating a relatively high difference between the expected and actual sales values.
This suggests that the model's sales forecast is inaccurate.


2. The RMSE value is 1.5181673918863379, representing the average deviation between the predicted and actual sales values.
 A high RMSE indicates inaccurate predictions and a significant difference between the predicted and actual values.


3. The R-squared value is 0.9169424999582725, indicating the model's ability to explain sales variations.
Values close to 1 indicate a good fit, and in this case, the model explains approximately 91.7% of the sales variance.


the model demonstrates acceptable performance in sales prediction based on the high R-squared value.
further analysis and understanding of the data and model context are necessary before making final conclusions.
 It may be beneficial to compare the model's performance with other models or
 explore additional improvements to enhance prediction accuracy.
""")



#Solutions include:

# collecting more data related to sales, and expanding the range of variables used in the model.
#This can help improve forecast accuracy and reduce the difference between predicted and actual values.


The goal of the code is to build a machine learning model to predict sales and thus to an advertising data set.


Mean squared error (MSE): 2.3048322297869657
Root Mean Squared Error (RMSE): 1.5181673918863379
R-squared (R2-score): 0.9169424999582725
Total iterations: 0

These results indicate that the model has a good ability to explain the variance in sales data, 
explaining about 91.7% of the variance in the data (R-squared).

There is a noticeable difference between the MSE value and the RMSE value, 
which indicates that the sales forecast provided by the model is inaccurate.

---------------------------------------------------------------more detel-----------------------------------


1. The MSE value is 2.3048322297869657, indicating a relatively high difference between the expected and actual sales values.
This suggests that the model's sales forecast is inaccurate.


2. The RMSE value is 1.5181673918863379, representing the average deviation between the predicted and actual sa